# launch scripts through SLURM 

The script in the cell below submits SLURM jobs running the requested `script`, over all file names in `MOLECULES_DIR`. 

The generated SBATCH scipts (`.job` files) are saved in the `jobs` folder and then submitted.
Output and error dumps are saved in the `out` folder.

In [7]:
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('..'))
from utils.load_lib import MOLECULES_DIR

origin_files = list(sorted(f[:-5] for f in os.listdir(MOLECULES_DIR)))
print(*enumerate(origin_files), sep = '\n')

(0, 'H,0,0,0;H,0.4053,0,0;H,-0.534,-0.0694,0;H,0.0902,-0.2637,-0.709')
(1, 'H,0,0,0;H,0.4063,0,0;H,0.3209,0.5626,0;H,0.3629,1.173,-0.0436')
(2, 'H,0,0,0;H,0.4066,0,0;H,0.2557,-0.7759,0;H,-0.0651,0.4897,0.3486')
(3, 'H,0,0,0;H,0.4084,0,0;H,0.8731,-0.1746,0;H,0.2598,0.2575,0.8294')
(4, 'H,0,0,0;H,0.4115,0,0;H,1.1163,-0.8134,0;H,0.4303,-0.4342,0.4398')
(5, 'H,0,0,0;H,0.413,0,0;H,-0.3051,1.1222,0;H,0.1887,0.2988,-0.4119')
(6, 'H,0,0,0;H,0.4132,0,0;H,0.3077,-0.7389,0;H,-0.5695,-1.0829,0.6991')
(7, 'H,0,0,0;H,0.4163,0,0;H,-0.5825,0.5499,0;H,-0.2441,0.9786,-0.7679')
(8, 'H,0,0,0;H,0.4187,0,0;H,0.8236,-0.9866,0;H,0.0328,-0.3853,0.7512')
(9, 'H,0,0,0;H,0.4187,0,0;H,0.9096,-0.2389,0;H,0.3747,-0.3106,0.2839')
(10, 'H,0,0,0;H,0.4199,0,0;H,-0.733,-0.4589,0;H,-0.2791,0.1134,-0.5084')
(11, 'H,0,0,0;H,0.4201,0,0;H,-0.1848,-0.8409,0;H,0.6355,-0.6911,-0.9798')
(12, 'H,0,0,0;H,0.4242,0,0;H,0.018,-1.2811,0;H,0.2568,-0.4245,-0.5248')
(13, 'H,0,0,0;H,0.4249,0,0;H,-0.0926,0.5247,0;H,0.0575,0.4332,0.7415')
(1

In [2]:
sys.path

['/Users/stefanopolla/git_repositories/leiden_repos/convoQC/convoQC/SLURM_working_directory',
 '/Users/stefanopolla/.ROOT/lib',
 '/usr/local/Caskroom/miniconda/base/lib/python37.zip',
 '/usr/local/Caskroom/miniconda/base/lib/python3.7',
 '/usr/local/Caskroom/miniconda/base/lib/python3.7/lib-dynload',
 '',
 '/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages',
 '/Users/stefanopolla/git_repositories/leiden_repos/QuantumDigitalCooling/qdclib/src',
 '/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/IPython/extensions',
 '/Users/stefanopolla/.ipython',
 '/Users/stefanopolla/git_repositories/leiden_repos/convoQC/convoQC']

In [11]:
#######################
### User parameters ###
#######################

script = 'optimize-ucc.py'
time = "00-10:00"  # format days-hh:mm
mem = "100MB"  # can use postfixes (MB, GB, ...)
partition = "compIntel"

additional_lines = [
    'source ~/.bashrc\n'
]


#####################################
### Create folders, files and run ###
#####################################

current_dir = os.path.abspath('.')
script = os.path.join(*os.path.split(current_dir)[:-1], 'scripts', script)
job_dir = 'jobs'
out_dir = 'out'

os.makedirs(job_dir, exist_ok=True)
os.makedirs(out_dir, exist_ok=True)

for idx, filename in enumerate(origin_files):
    job_name = 'H4-UCC-{}'.format(idx)
    job_file = os.path.join(job_dir, job_name+'.job')
    
    with open(job_file, 'wt') as fh:
        fh.writelines(
            ["#!/bin/bash\n",
            f"#SBATCH --job-name={job_name}\n",
            f"#SBATCH --output={os.path.join(out_dir, job_name+'.out')}\n",
            f"#SBATCH --error={os.path.join(out_dir, job_name+'.err')}\n",
            f"#SBATCH --time={time}\n",
            f"#SBATCH --mem={mem}\n",
            f"#SBATCH --partition={partition}\n",
            f"#SBATCH --mail-type=NONE\n",
            ] + additional_lines + [
            f"python -u {script} {filename}\n"]
        )

    os.system("sbatch %s" %job_file)